# Lecture 2 - 1

## Soviet planning problem

We are trying to 

The problem that we are trying to solve is 
\begin{align}
\min_{\pi \geq 0} \sum_{(x,y) \in \mathcal{A}} \pi_{xy} c_{xy} \\
\text{s.t. } \nabla^T \pi = n 
\end{align}

In [1]:
library("gurobi")
library("Matrix")

thePath = getwd()
data = as.matrix(read.csv(paste0(thePath, "/distances.csv"), sep = ",", header = TRUE))  # loads the data
head(data)
tail(data)

Loading required package: slam
Warning message:
"package 'slam' was built under R version 3.5.2"

X,Arkhangelsk,Yaroslavl.,Murom,Balakhonikha,Dzerzhinsk,Kishert.,Sverdlovsk,Artemovsk,Iledzhk,Dekonskaya,demand.
Agryz,NA,NA,NA,709,1064,693,NA,NA,NA,NA,2.0
Aleksandrov,NA,NA,NA,NA,397,NA,NA,1180,NA,NA,4.0
Almaznaya,NA,NA,NA,NA,NA,NA,NA,81,NA,65,1.5
Alchevskaya,NA,NA,NA,NA,NA,NA,NA,106,NA,114,4.0
Baku,NA,NA,NA,NA,NA,NA,NA,1554,NA,1563,10.0
Barybino,NA,NA,NA,NA,NA,NA,NA,985,NA,968,2.0


,X,Arkhangelsk,Yaroslavl.,Murom,Balakhonikha,Dzerzhinsk,Kishert.,Sverdlovsk,Artemovsk,Iledzhk,Dekonskaya,demand.
"[64,]",Chishmy,NA,NA,NA,1123,NA,773,NA,NA,889,NA,0.5
"[65,]",Shchigry,NA,NA,NA,NA,NA,NA,NA,566,NA,549,4.0
"[66,]",Yudino,NA,NA,NA,403,757,999,NA,NA,NA,NA,0.5
"[67,]",Yama,NA,NA,NA,NA,NA,NA,NA,44,NA,52,5.0
"[68,]",Yasinovataya,NA,NA,NA,NA,NA,NA,NA,85,NA,93,6.0
"[69,]",supply:,5,11.5,8.5,12,100,12,15,314,10,55,543.0


In [2]:
nsources = 68
ndests = 10
dists = matrix(as.numeric(data[1:68, 2:11]), 68, 10)
demand = matrix(as.numeric(data[1:68, 12]))
supply = matrix(as.numeric(data[69, 2:11]))
nonzeros = which(!is.na(dists))

nbNodes = nsources + ndests
nbArcs = length(nonzeros)

rows = (nonzeros - 1)%%nsources + 1
cols = (nonzeros - 1)%/%nsources + 1
c = dists[nonzeros]
arcs = cbind(rows, cols + nsources, c)

n = c(-demand, supply)
nameNodes = c(data[2:(nsources + 1)], dimnames(data)[[2]][2:11])

Construct node-incidence matrix:

In [3]:
Nabla = sparseMatrix(i = 1:nbArcs, j = arcs[, 1], dims = c(nbArcs, nbNodes), x = -1) + 
    sparseMatrix(i = 1:nbArcs, j = arcs[, 2], dims = c(nbArcs, nbNodes), x = 1)

Now let's solve using Gurobi:
* `A` = $\nabla^T$
* `obj` = $c$
* `sense` = '$=$'
* `rhs` = $n$
* `modelsense` = '$\min$'.

In [4]:
result = gurobi(list(A = t(Nabla), obj = c, modelsense = "min", rhs = n, sense = "=", 
    start = matrix(0, nbArcs, 1)), params = NULL)
pi = result$x
distance = result$objval

print(distance)

Optimize a model with 78 rows, 155 columns and 310 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 3e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-01, 3e+02]
Presolve removed 48 rows and 48 columns
Presolve time: 0.00s
Presolved: 30 rows, 107 columns, 214 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6658785e+05   4.252320e+02   0.000000e+00      0s
      37    3.9505200e+05   0.000000e+00   0.000000e+00      0s

Solved in 37 iterations and 0.00 seconds
Optimal objective  3.950520000e+05
[1] 395052
